# Transformer

In [1]:
import numpy as np

import respiration.utils as utils

tuned_models = {
    # '20240512_131641',
    '20240512_150151',
}

# Map model names to their paths
models = {}

manifests = []

for model_id in tuned_models:
    model_dir = utils.dir_path('models', 'transformer', model_id)

    manifest_path = utils.dir_path(model_dir, 'manifest.json')
    manifest = utils.read_json(manifest_path)
    best_model = manifest['trained_models'][-1]

    model_path = utils.join_paths(model_dir, best_model['model'])
    models[model_id] = model_path
    manifests.append(manifest)

utils.pretty_print(models)

In [2]:
from respiration.dataset import VitalCamSet

dataset = VitalCamSet()
scenarios = dataset.get_scenarios(['101_natural_lighting'])

device = utils.get_torch_device()
image_size = 256

In [3]:
import torch


def temporal_shifting_frames(frames: torch.Tensor) -> torch.Tensor:
    """
    Calculate the temporal shifting of the frames. This is done by calculating the difference between the frames and
    normalizing the result.
    """
    diff_frames = frames[1:] - frames[:-1]
    sum_frames = frames[1:] + frames[:-1]
    inputs = diff_frames / (sum_frames + 1e-7)
    inputs = (inputs - torch.mean(inputs)) / torch.std(inputs)
    return inputs


def temporal_shifting_signal(time_series: torch.Tensor) -> torch.Tensor:
    # Shift the signal that no negative values are present
    time_series = time_series - torch.min(time_series)

    # Calculate the difference between the time series
    diff = time_series[1:] - time_series[:-1]

    # Classify the differences into three classes: positive, zero, negative
    classes = torch.zeros((diff.shape[0], 2), dtype=torch.float32, device=device)
    classes[diff >= 0, 0] = 1.0
    classes[diff < 0, 1] = 1.0
    # classes[diff == 0, 1] = 1.0
    # classes[diff < 0, 2] = 1.0

    return classes


def signal_from_classes(classes: torch.Tensor) -> np.ndarray:
    signal = torch.zeros(classes.shape[0] + 1, dtype=torch.float32, device=device)
    signal[1:] = torch.argmax(classes, dim=1)

    # Make all 0 values to -1
    signal[signal == 0] = -1
    signal[signal == 1] = 1

    # for idx in range(1, signal.shape[0]):
    #     signal[idx] = signal[idx - 1] + signal[idx]

    return signal.cpu().numpy()

In [4]:
import torch
import datetime as dt

from tqdm.auto import tqdm
from vit_pytorch import ViT
from torchvision import transforms

predictions = []

for (subject, setting) in tqdm(reversed(scenarios)):
    print(f"Processing {subject} - {setting}")

    video_path = dataset.get_video_path(subject, setting)

    frames, _ = utils.read_video_rgb(video_path)
    preprocess = transforms.Compose([
        transforms.ToPILImage(mode='RGB'),
        transforms.Resize((image_size, image_size)),
        transforms.ToTensor()
    ])
    frames = torch.stack([preprocess(frame) for frame in frames], dim=0)
    frames = frames.to(device)
    frames = temporal_shifting_frames(frames)

    for (model_id, model_path) in models.items():
        print(f"--> Using {model_id} model")
        # Wrap modul in nn.DataParallel to fix the model loading issue
        model = ViT(
            image_size=image_size,
            patch_size=32,
            num_classes=2,
            dim=128,
            depth=6,
            heads=16,
            mlp_dim=2048
        ).to(device)
        model.load_state_dict(torch.load(model_path, map_location=device))
        model.eval()

        start = dt.datetime.now()

        with torch.no_grad():
            prediction = model(frames)

        signal = signal_from_classes(prediction)

        predictions.append({
            'model': model_id,
            'subject': subject,
            'setting': setting,
            'duration': dt.datetime.now() - start,
            'signal': signal.tolist(),
        })

    del frames
    break

In [5]:
# Save the predictions
import pandas as pd

predictions_df = pd.DataFrame(predictions)
signals = utils.dir_path('outputs', 'signals')
signals_path = utils.join_paths(signals, 'transformer_classifier_predictions.csv')
predictions_df.to_csv(signals_path, index=False)

In [6]:
import matplotlib.pyplot as plt
import respiration.analysis as analysis

subject = 'Proband26'
setting = '101_natural_lighting'

gt = dataset.get_breathing_signal(subject, setting)
gt = torch.tensor(gt, dtype=torch.float32, device=device)
xxx = temporal_shifting_signal(gt)
gt_signal = signal_from_classes(xxx)

compare = analysis.SignalComparator(gt.cpu().numpy(), gt_signal, 30)
compare.errors()

In [7]:
compare.signal_distances()

In [8]:
fig, axs = plt.subplots(2, 1, figsize=(20, 6))

# Add some space between the plots
fig.subplots_adjust(hspace=0.5)

axs[0].plot(compare.ground_truth, label='Ground Truth')
axs[0].set_title('Ground Truth')
axs[0].legend()

axs[1].plot(compare.prediction, label='Predicted')
axs[1].set_title('Predicted')

plt.show()

In [9]:
prediction_subject = predictions_df[(predictions_df['subject'] == subject) &
                                    (predictions_df['setting'] == setting)]

predicted_signal = np.array(prediction_subject['signal'].values[0])
compare = analysis.SignalComparator(predicted_signal, gt_signal, 30)
compare.errors()

In [10]:
fig, axs = plt.subplots(2, 1, figsize=(20, 6))

axs[0].plot(compare.ground_truth, label='Ground Truth')
axs[0].set_title('Ground Truth vs Predicted Signal')
axs[0].legend()

axs[1].plot(predicted_signal, label='Predicted')
axs[1].set_title('Ground Truth Signal Classes')

plt.show()

In [11]:
diff_signal = predicted_signal[1:] - predicted_signal[:-1]
diff_signal[450:550]

In [14]:
# Count the number of -1 and 1 in the signal
np.unique(diff_signal, return_counts=True)